In [8]:
from icalendar import Calendar, Event
import pandas as pd
import re

In [9]:
ical_file = 'schedule.ics'
with open(ical_file) as f:
    ical_text = f.read()
cal = Calendar.from_ical(ical_text)
w = cal.walk()
w = w[1:] # cal is the first item for some reason

In [56]:
def event_to_dict(e):
    event_dict = {key.lower() : clean_bytes(e.decoded(key)) for key in e}
    del event_dict['uid']
    del event_dict['sequence']
    desc_dict = extract_desc(event_dict['description'])
    for key in desc_dict:
        event_dict[key] = desc_dict[key]
    del event_dict['description']
    
    event_dict['person'] = extract_person(event_dict['summary'], event_dict['shift'])

    return event_dict

def extract_person(summary_str, shift_code):
    summ_str_split = summary_str.split(' ')
    return summ_str_split[-2] + ' ' + summ_str_split[-1]
    # return summary_str.split(shift_code + ' ')[-1]

def extract_desc(desc):
    lines = desc.split('\n')
    desc_dict = {}
    for line in lines:
        colon_loc = line.find(':')
        key = line[:colon_loc]
        val = line[colon_loc+1:]
        desc_dict[key.lower()] = val[1:]
    return desc_dict

def clean_bytes(b):
    if type(b) == bytes:
        return b.decode('utf8')
    else:
        return b

In [57]:
event_dicts = [event_to_dict(e) for e in w]
# for i, e in enumerate(w):
#     try:
#         # print(i)
#         event_to_dict(e)
#     except Exception as e:
#         print(i)
#         raise e

In [58]:
df = pd.DataFrame(event_dicts)

In [59]:
df['shift type'].value_counts()

PGY 1 (UH, UI, SH, SI)          892
EM 4 (UJ, UL, SJ, SL)           384
EM 23 (SA, SB, SD)              320
EM 3 (UX, UY, UZ)               319
EM 2,3,4 (EC3 Shifts)           224
EM 2 (UK, UC, UG)               224
Jeopardy                        219
EM 34 (HPed, UPed)               32
EM 234 - SF                      19
Manual Shifts (TS/Anes/Peds)      7
EM 234 - Extra (UR, SC)           6
EM 23 - Extra (UF)                3
EM 12 (T3)                        1
Track - EMS                       1
Name: shift type, dtype: int64

In [62]:

df.person.value_counts()

M Griffith         59
E Ager             58
M Newton           55
M Abdel-Hamid      55
M Mitchell         53
                   ..
Z Kramer            2
N Makki             2
T Kolodziejczyk     2
N Abdi              1
K Fraser            1
Name: person, Length: 99, dtype: int64

In [60]:
df.sort_values(['dtstart','facility'])

,dtstamp,dtstart,dtend,summary,group,facility,shift,shift type,person
0,2021-08-16 19:19:01+00:00,2021-07-01 06:00:00-04:00,2021-07-01 16:00:00-04:00,SJ SA C Agubosim,UM/St. Joe's - EM,St. Joseph Mercy Hospital,SA,"EM 23 (SA, SB, SD)",C Agubosim
1,2021-08-16 19:19:01+00:00,2021-07-01 06:00:00-04:00,2021-07-01 16:00:00-04:00,SJ SH M Davis,UM/St. Joe's - EM,St. Joseph Mercy Hospital,SH,"PGY 1 (UH, UI, SH, SI)",M Davis
8,2021-08-16 19:19:01+00:00,2021-07-01 07:00:00-04:00,2021-07-01 16:00:00-04:00,SJ SJ K Grace,UM/St. Joe's - EM,St. Joseph Mercy Hospital,SJ,"EM 4 (UJ, UL, SJ, SL)",K Grace
2,2021-08-16 19:19:01+00:00,2021-07-01 07:00:00-04:00,2021-07-01 16:00:00-04:00,UM UH L Kruger,UM/St. Joe's - EM,University of Michigan,UH,"PGY 1 (UH, UI, SH, SI)",L Kruger
3,2021-08-16 19:19:01+00:00,2021-07-01 07:00:00-04:00,2021-07-01 16:00:00-04:00,UM UH V Lin,UM/St. Joe's - EM,University of Michigan,UH,"PGY 1 (UH, UI, SH, SI)",V Lin
...,...,...,...,...,...,...,...,...,...
2646,2021-08-16 19:19:02+00:00,2021-10-20 20:00:00-04:00,2021-10-21 04:00:00-04:00,SJ SE N Rosculet,UM/St. Joe's - EM,St. Joseph Mercy Hospital,SE,"EM 234 - Extra (UR, SC)",N Rosculet
2648,2021-08-16 19:19:02+00:00,2021-10-20 22:00:00-04:00,2021-10-21 08:00:00-04:00,SJ SD N Valentini,UM/St. Joe's - EM,St. Joseph Mercy Hospital,SD,"EM 23 (SA, SB, SD)",N Valentini
2647,2021-08-16 19:19:02+00:00,2021-10-20 22:00:00-04:00,2021-10-21 07:00:00-04:00,UM UG A Krumheuer,UM/St. Joe's - EM,University of Michigan,UG,"EM 2 (UK, UC, UG)",A Krumheuer
2649,2021-08-16 19:19:02+00:00,2021-10-20 23:00:00-04:00,2021-10-21 07:00:00-04:00,UM UZ M Grzywinski,UM/St. Joe's - EM,University of Michigan,UZ,"EM 3 (UX, UY, UZ)",M Grzywinski


In [7]:
blah = event_to_dict(w[1])
print(blah['DESCRIPTION'])

Group: UM/St. Joe's - EM
Facility: St. Joseph Mercy Hospital
Shift: SH
Shift Type: PGY 1 (UH, UI, SH, SI)


In [ ]:
blah = event_to_dict(w[1])
blah['UID']

In [ ]:
type(blah['UID'])

In [ ]:
for e in w:
    print(e.property_items())